# Main App

In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
#|export
import pandas as pd
from flask import Flask, redirect, request, render_template

In [26]:

from IPython.display import display_html

In [27]:
people = [
    {
        "firstname": "John",
        "lastname": "Doe",
        "phone": "555-1234",
        "email": "john.doe@example.com"
    },
    {
        "firstname": "Jane",
        "lastname": "Smith",
        "phone": "555-5678",
        "email": "jane.smith@example.com"
    },
    {
        "firstname": "Alice",
        "lastname": "Johnson",
        "phone": "555-8765",
        "email": "alice.johnson@example.com"
    },
    {
        "firstname": "Bob",
        "lastname": "Williams",
        "phone": "555-4321",
        "email": "bob.williams@example.com"
    },
    {
        "firstname": "Charlie",
        "lastname": "Brown",
        "phone": "555-9876",
        "email": "charlie.brown@example.com"
    },
    {
        "firstname": "Diana",
        "lastname": "Miller",
        "phone": "555-6543",
        "email": "diana.miller@example.com"
    },
    {
        "firstname": "Edward",
        "lastname": "Davis",
        "phone": "555-3456",
        "email": "edward.davis@example.com"
    },
    {
        "firstname": "Fiona",
        "lastname": "Garcia",
        "phone": "555-6789",
        "email": "fiona.garcia@example.com"
    },
    {
        "firstname": "George",
        "lastname": "Martinez",
        "phone": "555-7890",
        "email": "george.martinez@example.com"
    },
    {
        "firstname": "Hannah",
        "lastname": "Rodriguez",
        "phone": "555-8901",
        "email": "hannah.rodriguez@example.com"
    }
]

# Writing the JSON data to a file
file_path = 'contactdb.json'
# df = pd.read_json(file_path)
df = pd.DataFrame()
df = df.from_dict(people)
df['id'] = df.index
df.to_json(file_path, orient='records')

In [4]:
db = pd.read_json('contactdb.json')
db.to_dict('records')

[{'firstname': 'John',
  'lastname': 'Doe',
  'phone': '555-1234',
  'email': 'john.doe@example.com',
  'id': 0},
 {'firstname': 'Jane',
  'lastname': 'Smith',
  'phone': '555-5678',
  'email': 'jane.smith@example.com',
  'id': 1},
 {'firstname': 'Alice',
  'lastname': 'Johnson',
  'phone': '555-8765',
  'email': 'alice.johnson@example.com',
  'id': 2},
 {'firstname': 'Bob',
  'lastname': 'Williams',
  'phone': '555-4321',
  'email': 'bob.williams@example.com',
  'id': 3},
 {'firstname': 'Charlie',
  'lastname': 'Brown',
  'phone': '555-9876',
  'email': 'charlie.brown@example.com',
  'id': 4},
 {'firstname': 'Diana',
  'lastname': 'Miller',
  'phone': '555-6543',
  'email': 'diana.miller@example.com',
  'id': 5},
 {'firstname': 'Edward',
  'lastname': 'Davis',
  'phone': '555-3456',
  'email': 'edward.davis@example.com',
  'id': 6},
 {'firstname': 'Fiona',
  'lastname': 'Garcia',
  'phone': '555-6789',
  'email': 'fiona.garcia@example.com',
  'id': 7},
 {'firstname': 'George',
  'last

In [5]:
q = '1234'
# db[db['firstname'].str.lower().str.contains(q)]
predicate_firstname = db['firstname'].str.lower().str.contains(q.lower())
predicate_lastname = db['lastname'].str.lower().str.contains(q.lower())
predicate_phone = db['phone'].str.lower().str.contains(q.lower())
predicate_email= db['email'].str.lower().str.contains(q.lower())
db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]

,firstname,lastname,phone,email,id
0,John,Doe,555-1234,john.doe@example.com,0


In [6]:
db.columns

a={'firstname': 'Rahul',
   'lastname': 'Saraf',
    'phone':'7347306799',
    'email': 'rahuketu86'}
a['id'] = len(db)
# db.loc[len(db)] = 
a
db.loc[len(db), a.keys()] = a.values()

In [7]:
db

,firstname,lastname,phone,email,id
0,John,Doe,555-1234,john.doe@example.com,0.0
1,Jane,Smith,555-5678,jane.smith@example.com,1.0
2,Alice,Johnson,555-8765,alice.johnson@example.com,2.0
3,Bob,Williams,555-4321,bob.williams@example.com,3.0
4,Charlie,Brown,555-9876,charlie.brown@example.com,4.0
5,Diana,Miller,555-6543,diana.miller@example.com,5.0
6,Edward,Davis,555-3456,edward.davis@example.com,6.0
7,Fiona,Garcia,555-6789,fiona.garcia@example.com,7.0
8,George,Martinez,555-7890,george.martinez@example.com,8.0
9,Hannah,Rodriguez,555-8901,hannah.rodriguez@example.com,9.0


In [8]:
#|export
class Contact(object):
    def __init__(self) -> None:
        self.refresh()

    def search(self, q):
        predicate_firstname = self.db['firstname'].str.lower().str.contains(q.lower())
        predicate_lastname = self.db['lastname'].str.lower().str.contains(q.lower())
        predicate_phone = self.db['phone'].str.lower().str.contains(q.lower())
        predicate_email= self.db['email'].str.lower().str.contains(q.lower())
        filter_df = self.db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]
        return filter_df.to_dict('records')
    
    def refresh(self):
        self.file_path = 'contactdb.json'
        self.db = pd.read_json(self.file_path)


    def all(self):
        return self.db.to_dict('records')
    
    def get(self, id):
        contact = self.db[self.db['id']==id]
        # print(contact)
        return contact.to_dict('records')[0]
    
    def add(self, firstname, lastname, phone, email):
        a={'firstname': firstname,
           'lastname': lastname,
           'phone':phone,
           'email': email}
        a['id'] = len(self.db)
        self.db.loc[len(self.db), a.keys()] = a.values()
        self.db.to_json(file_path, orient='records')
        self.refresh()

    def delete(self, id):
        self.db.drop(index=id, inplace=True, errors='raise')
        self.db.to_json(file_path, orient='records')
        self.refresh()

    def edit(self, id, firstname, lastname, phone, email):
        a={'firstname': firstname,
           'lastname': lastname,
           'phone':phone,
           'email': email,
           'id': id}
        self.db.loc[id, a.keys()] = a.values()
        self.db.to_json(file_path, orient='records')
        self.refresh()


In [10]:
c = Contact()
c.db.loc[2].to_dict().values()

dict_values(['Alice', 'Johnson', '555-8765', 'alice.johnson@example.com', 2])

In [11]:
new_vals = [2, 'Alice', 'Johnson', '555-8767', 'alice.johnson@example.com']
c.edit(*new_vals)
c.get(2)

{'firstname': 'Alice',
 'lastname': 'Johnson',
 'phone': '555-8767',
 'email': 'alice.johnson@example.com',
 'id': 2}

In [13]:
c = Contact()
a_dict = c.db.loc[0].to_dict()
a_dict.pop('id', None); a_dict
c.delete(0)

c.add(*list(a_dict.values()))
c.all()




[{'firstname': 'Alice',
  'lastname': 'Johnson',
  'phone': '555-8767',
  'email': 'alice.johnson@example.com',
  'id': 2},
 {'firstname': 'Bob',
  'lastname': 'Williams',
  'phone': '555-4321',
  'email': 'bob.williams@example.com',
  'id': 3},
 {'firstname': 'Charlie',
  'lastname': 'Brown',
  'phone': '555-9876',
  'email': 'charlie.brown@example.com',
  'id': 4},
 {'firstname': 'Diana',
  'lastname': 'Miller',
  'phone': '555-6543',
  'email': 'diana.miller@example.com',
  'id': 5},
 {'firstname': 'Edward',
  'lastname': 'Davis',
  'phone': '555-3456',
  'email': 'edward.davis@example.com',
  'id': 6},
 {'firstname': 'Fiona',
  'lastname': 'Garcia',
  'phone': '555-6789',
  'email': 'fiona.garcia@example.com',
  'id': 7},
 {'firstname': 'George',
  'lastname': 'Martinez',
  'phone': '555-7890',
  'email': 'george.martinez@example.com',
  'id': 8},
 {'firstname': 'Hannah',
  'lastname': 'Rodriguez',
  'phone': '555-8901',
  'email': 'hannah.rodriguez@example.com',
  'id': 9},
 {'fir

In [14]:
#|export
app = Flask(__name__)

In [15]:
#|export
@app.get("/")
def index():
    return redirect("/contacts")

In [16]:
#|export
@app.get("/contacts")
def contacts():
    search = request.args.get("q")
    contact_set = None
    if search is not None: contact_set = Contact().search(search)
    else: contact_set = Contact().all()
    # print(contact_set)
    return render_template("index.html", contact_set=contact_set)

In [17]:
Contact().db[df['id']==1].to_dict('records')[0]

{'firstname': 'Bob',
 'lastname': 'Williams',
 'phone': '555-4321',
 'email': 'bob.williams@example.com',
 'id': 3}

In [18]:
#|export
@app.get("/contacts/<int:id>")
def view(id:int):
    return render_template("view.html", contact=Contact().get(id))

In [19]:
with app.app_context():
    print(view(2))

<!doctype html>
<html>
    <head>
        <title>Contact.App</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@picocss/pico@2/css/pico.min.css">
    </head>
    <body>
        <main class="container">
            <h1> CONTACTS.APP</h1>
            <h2> A demo contact application </h2>
            <hr>
            
    <article>
        <h1>Alice Johnson </h1>
        <div role="group">
            <div>555-8767</div>
            <div>alice.johnson@example.com</div>
        </div>
        <footer>
            <a href="/contacts" role="button">Back</a>
        </footer>
    </article>

        </main>
    </body>
</html>


In [20]:
Contact().get(2)

{'firstname': 'Alice',
 'lastname': 'Johnson',
 'phone': '555-8767',
 'email': 'alice.johnson@example.com',
 'id': 2}

In [21]:
#|export
@app.put("/contacts/<int:id>/edit")
def edit(id):
    return "Edit View"

In [22]:
#|export
@app.post("/contacts/new")
def new():
    return "New View"

In [24]:
#|eval:false
from nbdev.export import nb_export
nb_export("01_main.ipynb", lib_path=".", name='main')